In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn import metrics
from sklearn import svm
import pandas as pd
import numpy as np
import sys
import os

In [20]:
is_vae = True

In [21]:
# Just choose the name of the dataset directory
DATA_DIR = '/Users/tomas/Documents/FEUP/Tese/data/ml-20m/processed_70_10_20'
if is_vae:
    PARSE_DATA_DIR = os.path.join(DATA_DIR, 'embeddings/vae')
else:
    PARSE_DATA_DIR = os.path.join(DATA_DIR, 'embeddings/cdae')

In [62]:
PARSE_DATA_DIR = os.path.join(DATA_DIR, 'embeddings/als')

In [63]:
file = 'metadataset_k_20.csv'
os.path.join(PARSE_DATA_DIR, file )

'/Users/tomas/Documents/FEUP/Tese/data/ml-20m/processed_70_10_20/embeddings/als/metadataset_k_20.csv'

In [64]:
#read in the data using pandas
metadataset = pd.read_csv(os.path.join(PARSE_DATA_DIR, file ))
#metadataset = metadataset[metadataset.first_place != 'zeroes']
metadataset.head()

,original_id,0,1,2,3,4,5,6,7,8,...,191,192,193,194,195,196,197,198,199,first_place
0,1,-0.058967,0.229568,-0.707001,0.540223,-0.110984,0.219160,0.191912,0.397641,0.412974,...,-0.166100,0.179952,0.182467,0.175274,-0.002350,0.520412,0.183931,-0.089415,0.294081,zeroes
1,2,0.053650,0.003824,-0.151874,0.223005,-0.076169,0.024204,-0.226282,-0.275735,-0.287022,...,0.015414,0.253139,0.023921,-0.157192,0.178530,0.095084,-0.079894,-0.121272,-0.002467,als_ndcg
2,3,-0.202143,-0.226526,-0.648742,0.234206,0.170624,0.496507,-0.282112,0.455193,-0.109085,...,-0.321206,0.287338,-0.166969,0.110853,0.312051,0.262411,0.061551,0.050908,-0.211964,most_popular_ndcg
3,4,-0.155596,0.016257,0.255050,0.119575,-0.315419,-0.087295,-0.286379,-0.054195,-0.147150,...,-0.055110,-0.010709,0.287566,0.097178,0.251507,-0.101567,0.070255,0.004031,-0.131053,zeroes
4,5,-0.017234,0.420561,-0.288330,0.445641,-0.164420,0.055104,-0.175972,-0.130913,-0.084127,...,0.161235,0.038432,0.312712,-0.041273,0.328645,0.034584,0.055259,-0.273009,-0.147063,most_popular_ndcg


### Encode Target

In [65]:
#als:0
#bpr:1
#lmf:2
#most_pop_3
#zeros:4
target_pre = metadataset['first_place'].values 
label_encoder = LabelEncoder()
target = label_encoder.fit_transform(target_pre)

keys = label_encoder.classes_
values = label_encoder.transform(keys)
labels = dict(zip(values,keys))

### Normalization Inputs

In [66]:
normalize = False

In [67]:
if normalize:
  #---- SET INPUTS -----
  scaler = StandardScaler()
  #Compute the mean and std to be used for later scaling.
  scaler.fit(metadataset.drop(columns=['first_place','original_id']))
  # Perform standardization by centering and scaling
  inputs_transform = scaler.transform(metadataset.drop(columns=['first_place','original_id']))
  inputs = pd.DataFrame(inputs_transform)
  inputs.head()
else:
  inputs = metadataset.drop(columns=['first_place','original_id'])

### Model

In [68]:
kf = KFold(n_splits=5)
kf.get_n_splits()
print(kf)

KFold(n_splits=5, random_state=None, shuffle=False)


In [69]:
is_smote = False

In [77]:
params ={'max_features': 'auto', 'n_estimators': 1000, 'bootstrap': True, 'max_depth': 150}


In [78]:
i = 1 
base_impact_with_zeroes = []
base_impact_without_zeroes_most = []
base_impact_without_zeroes_best = []
matrix = []
reports = []

base_impact_with_zeroes_without_true_zeroes = []
base_impact_without_zeroes_most_without_true_zeroes = []
base_impact_without_zeroes_best_without_true_zeroes = []

confusion_without_true_zeroes = []
reports_without_true_zeroes = []
for train_index, test_index in kf.split(inputs):
    print('iteration: ', i)
    #get data fold
    X_train, X_test = inputs.iloc[train_index], inputs.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    #start model 
    print('fit')
    clf = RandomForestClassifier(
        random_state=0,
        bootstrap=params['bootstrap'],
        max_depth=params['max_depth'],
        max_features=params['max_features'],
        n_estimators=params['n_estimators'], verbose=100, n_jobs=-1)
    
    
    if is_smote:
        print('dataset shape %s' % Counter(y_train))
        sm = SMOTE(random_state=42,n_jobs=4)
        X_train_re, y_train_re = sm.fit_resample(X_train, y_train)
        print('Resampled dataset shape %s' % Counter(y_train_re))

        clf.fit(X_train_re, y_train_re)
        print('predict')
    else:
        clf.fit(X_train, y_train)
        
    y_pred = clf.predict(X_test)
    
    
      #Remove users that are zeroes 
    data = {'real':  y_test,
        'pred': y_pred,
        'original_id':metadataset.iloc[test_index]['original_id'].values
    }
    df = pd.DataFrame (data, columns = ['real','pred','original_id'])
    df = df[df.real != 4]
    
    
    report = classification_report(y_test, 
                               y_pred, 
                               target_names=np.unique(metadataset['first_place'].values),
                              output_dict=True)
    reports.append(report)
    
    bl_zeroes, bl_no_zeroes_most, bl_no_zeroes_best = base_level_eval(metadataset.iloc[test_index]['original_id'].values,
             list(label_encoder.inverse_transform(y_pred)))

    base_impact_with_zeroes.append(bl_zeroes)
    base_impact_without_zeroes_most.append(bl_no_zeroes_most)
    base_impact_without_zeroes_best.append(bl_no_zeroes_best)

    confusion = confusion_matrix(y_test,y_pred)
    matrix.append(confusion)
    np.set_printoptions(suppress=True)
    print('end: ', i)
    i+=1

iteration:  1
fit
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
building tree 1 of 1000building tree 2 of 1000
building tree 3 of 1000building tree 4 of 1000


building tree 5 of 1000[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.4s

building tree 6 of 1000[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.4s

building tree 7 of 1000[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    4.4s

building tree 8 of 1000[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    4.5s

building tree 9 of 1000[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    8.7s

building tree 10 of 1000[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    8.7s

building tree 11 of 1000[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    8.8s

building tree 12 of 1000[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    9.0s

building tree 13 of 1000[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   12.7s

building tree 14 of 1000[Pa

building tree 97 of 1000[Parallel(n_jobs=-1)]: Done  93 tasks      | elapsed:  1.5min

building tree 98 of 1000[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  1.5min

building tree 99 of 1000[Parallel(n_jobs=-1)]: Done  95 tasks      | elapsed:  1.5min

building tree 100 of 1000[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:  1.5min

building tree 101 of 1000[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  1.5min

building tree 102 of 1000[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  1.5min

building tree 103 of 1000[Parallel(n_jobs=-1)]: Done  99 tasks      | elapsed:  1.5min

building tree 104 of 1000[Parallel(n_jobs=-1)]: Done 100 tasks      | elapsed:  1.6min

building tree 105 of 1000[Parallel(n_jobs=-1)]: Done 101 tasks      | elapsed:  1.6min

building tree 106 of 1000[Parallel(n_jobs=-1)]: Done 102 tasks      | elapsed:  1.6min

building tree 107 of 1000[Parallel(n_jobs=-1)]: Done 103 tasks      | elapsed:  1.6min

building tree 108 of 1000[Parallel(

building tree 192 of 1000[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:  3.1min

building tree 193 of 1000[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:  3.2min

building tree 194 of 1000[Parallel(n_jobs=-1)]: Done 190 tasks      | elapsed:  3.2min

building tree 195 of 1000[Parallel(n_jobs=-1)]: Done 191 tasks      | elapsed:  3.2min

building tree 196 of 1000[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.2min

building tree 197 of 1000[Parallel(n_jobs=-1)]: Done 193 tasks      | elapsed:  3.2min

building tree 198 of 1000[Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed:  3.2min

building tree 199 of 1000[Parallel(n_jobs=-1)]: Done 195 tasks      | elapsed:  3.2min

building tree 200 of 1000[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  3.2min

building tree 201 of 1000[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed:  3.3min

building tree 202 of 1000[Parallel(n_jobs=-1)]: Done 198 tasks      | elapsed:  3.3min

building tree 203 of 1000[Parall

building tree 286 of 1000[Parallel(n_jobs=-1)]: Done 282 tasks      | elapsed:  4.6min

building tree 287 of 1000[Parallel(n_jobs=-1)]: Done 283 tasks      | elapsed:  4.7min

building tree 288 of 1000[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  4.7min

building tree 289 of 1000[Parallel(n_jobs=-1)]: Done 285 tasks      | elapsed:  4.7min

building tree 290 of 1000[Parallel(n_jobs=-1)]: Done 286 tasks      | elapsed:  4.7min

building tree 291 of 1000[Parallel(n_jobs=-1)]: Done 287 tasks      | elapsed:  4.7min

building tree 292 of 1000[Parallel(n_jobs=-1)]: Done 288 tasks      | elapsed:  4.7min

building tree 293 of 1000[Parallel(n_jobs=-1)]: Done 289 tasks      | elapsed:  4.8min

building tree 294 of 1000[Parallel(n_jobs=-1)]: Done 290 tasks      | elapsed:  4.8min

building tree 295 of 1000[Parallel(n_jobs=-1)]: Done 291 tasks      | elapsed:  4.8min

building tree 296 of 1000[Parallel(n_jobs=-1)]: Done 292 tasks      | elapsed:  4.8min

building tree 297 of 1000[Parall

building tree 380 of 1000[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:  6.0min

building tree 381 of 1000[Parallel(n_jobs=-1)]: Done 377 tasks      | elapsed:  6.1min

building tree 382 of 1000[Parallel(n_jobs=-1)]: Done 378 tasks      | elapsed:  6.1min

building tree 383 of 1000[Parallel(n_jobs=-1)]: Done 379 tasks      | elapsed:  6.1min

building tree 384 of 1000[Parallel(n_jobs=-1)]: Done 380 tasks      | elapsed:  6.1min

building tree 385 of 1000[Parallel(n_jobs=-1)]: Done 381 tasks      | elapsed:  6.1min

building tree 386 of 1000[Parallel(n_jobs=-1)]: Done 382 tasks      | elapsed:  6.1min

building tree 387 of 1000[Parallel(n_jobs=-1)]: Done 383 tasks      | elapsed:  6.1min

building tree 388 of 1000[Parallel(n_jobs=-1)]: Done 384 tasks      | elapsed:  6.1min

building tree 389 of 1000[Parallel(n_jobs=-1)]: Done 385 tasks      | elapsed:  6.2min

building tree 390 of 1000[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:  6.2min

building tree 391 of 1000[Parall

...

[Parallel(n_jobs=4)]: Done 431 tasks      | elapsed:    4.0s
[Parallel(n_jobs=4)]: Done 432 tasks      | elapsed:    4.0s
[Parallel(n_jobs=4)]: Done 433 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 434 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 435 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 436 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 437 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 438 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 439 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 440 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 441 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 443 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 444 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 445 tasks      | elapsed:    4.2s
[Parallel(n_jobs=4)]: Done 446 tasks      | elapsed:    4.2s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 568 tasks      | elapsed:    5.3s
[Parallel(n_jobs=4)]: Done 569 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 570 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 571 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 572 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 573 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 574 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 575 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 576 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 577 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 578 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 579 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 580 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 581 tasks      | elapsed:    5.5s
[Parallel(n_jobs=4)]: Done 582 tasks      | elapsed:    5.5s
[Parallel(n_jobs=4)]: Done 583 tasks      | elapsed:    5.5s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 717 tasks      | elapsed:    6.8s
[Parallel(n_jobs=4)]: Done 718 tasks      | elapsed:    6.8s
[Parallel(n_jobs=4)]: Done 719 tasks      | elapsed:    6.8s
[Parallel(n_jobs=4)]: Done 720 tasks      | elapsed:    6.8s
[Parallel(n_jobs=4)]: Done 721 tasks      | elapsed:    6.8s
[Parallel(n_jobs=4)]: Done 722 tasks      | elapsed:    6.9s
[Parallel(n_jobs=4)]: Done 723 tasks      | elapsed:    6.9s
[Parallel(n_jobs=4)]: Done 724 tasks      | elapsed:    6.9s
[Parallel(n_jobs=4)]: Done 725 tasks      | elapsed:    6.9s
[Parallel(n_jobs=4)]: Done 726 tasks      | elapsed:    6.9s
[Parallel(n_jobs=4)]: Done 727 tasks      | elapsed:    6.9s
[Parallel(n_jobs=4)]: Done 728 tasks      | elapsed:    6.9s
[Parallel(n_jobs=4)]: Done 729 tasks      | elapsed:    6.9s
[Parallel(n_jobs=4)]: Done 730 tasks      | elapsed:    7.0s
[Parallel(n_jobs=4)]: Done 731 tasks      | elapsed:    7.0s
[Parallel(n_jobs=4)]: Done 732 tasks      | elapsed:    7.0s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 852 tasks      | elapsed:    8.5s
[Parallel(n_jobs=4)]: Done 853 tasks      | elapsed:    8.5s
[Parallel(n_jobs=4)]: Done 854 tasks      | elapsed:    8.5s
[Parallel(n_jobs=4)]: Done 855 tasks      | elapsed:    8.5s
[Parallel(n_jobs=4)]: Done 856 tasks      | elapsed:    8.5s
[Parallel(n_jobs=4)]: Done 857 tasks      | elapsed:    8.5s
[Parallel(n_jobs=4)]: Done 858 tasks      | elapsed:    8.5s
[Parallel(n_jobs=4)]: Done 859 tasks      | elapsed:    8.5s
[Parallel(n_jobs=4)]: Done 860 tasks      | elapsed:    8.6s
[Parallel(n_jobs=4)]: Done 861 tasks      | elapsed:    8.6s
[Parallel(n_jobs=4)]: Done 862 tasks      | elapsed:    8.6s
[Parallel(n_jobs=4)]: Done 863 tasks      | elapsed:    8.6s
[Parallel(n_jobs=4)]: Done 864 tasks      | elapsed:    8.6s
[Parallel(n_jobs=4)]: Done 865 tasks      | elapsed:    8.6s
[Parallel(n_jobs=4)]: Done 866 tasks      | elapsed:    8.6s
[Parallel(n_jobs=4)]: Done 867 tasks      | elapsed:    8.6s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 993 tasks      | elapsed:   10.2s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:   10.2s finished
starting base_level_eval
same 0.6971233105555283 0.6971233105555283
same 0.0 0.0
same 0.3243991050595311 0.3243991050595311
same 0.36990623328406574 0.36990623328406574
0 0.5
same 0.5 0.5
0.6051730872497956 0.0
%users that predicted class ndcg is better than ALS  0.08604678047088744
%users that predicted class ndcg is same than ALS 0.7987746136950407
%users that predicted class ndcg is worst than ALS 0.1151786058340719
end:  5


In [80]:
print('base level impact zeroes', np.mean(base_impact_with_zeroes))
print('base level impact  whithout zeroes, replaced most_pop', np.mean(base_impact_without_zeroes_most))
print('base level impact  whithout zeroes, replaced best', np.mean(base_impact_without_zeroes_best))
avg_reports = report_average(reports)
print_report(avg_reports)
print_confusion(np.mean( np.array(matrix),axis=0 ),labels)

base level impact zeroes 0.25494582023385987
base level impact  whithout zeroes, replaced most_pop 0.2750803748593794
base level impact  whithout zeroes, replaced best 0.297883445474696
+-------------------+---------------------+-----------------------+----------------------+
|     Algorithm     |      Precision      |         Recall        |          F1          |
+-------------------+---------------------+-----------------------+----------------------+
|      als_ndcg     | 0.29538531274818003 |   0.7409170937483065  | 0.42234589770773556  |
|      bpr_ndcg     |  0.3529631675205021 |  0.11008932015340855  | 0.16781897450410052  |
|      lmf_ndcg     | 0.17321637426900585 | 0.0007790627258362567 | 0.001551094511526192 |
| most_popular_ndcg |   0.32877795952314  |  0.19630102550962383  | 0.24567521596916936  |
|       zeroes      | 0.39502111265192924 |   0.3418234408193114  |  0.3664714181046452  |
|        ---        |         ---         |          ---          |         ---       

In [76]:
print('base level impact zeroes', np.mean(base_impact_with_zeroes_without_true_zeroes))
print('base level impact  whithout zeroes, replaced most_pop', np.mean(base_impact_without_zeroes_most_without_true_zeroes))
print('base level impact  whithout zeroes, replaced best', np.mean(base_impact_without_zeroes_best_without_true_zeroes))

avg_reports = report_average(reports_without_true_zeroes)
print_report(avg_reports)
print_confusion(np.mean( np.array(confusion_without_true_zeroes),axis=0 ),labels)

base level impact zeroes nan
base level impact  whithout zeroes, replaced most_pop nan
base level impact  whithout zeroes, replaced best nan


/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


IndexError: list index out of range

#### print

In [70]:

def print_confusion(values, classes):
    from prettytable import PrettyTable
    x = PrettyTable()
    print(classes)

    names = []
    names.append('algorithm')
    names = names + list(classes.values())

    x.field_names = names

    i = 0
    for row in values:
        #row = np.array(row)
        r = []
        r.append(classes[i])
        row = r + list(row)
        #r.append(classes[i])
        #r = r + row
        #row = np.insert(row,0,'als')
        x.add_row(row)
        #r  = np.concatenate(csses[i],row[])
        i +=1
    print(x)
def report_average(reports):
    mean_dict = dict()
    for label in reports[0].keys():
        dictionary = dict()

        if label in 'accuracy':
            mean_dict[label] = sum(d[label] for d in reports) / len(reports)
            continue

        for key in reports[0][label].keys():
            dictionary[key] = sum(d[label][key] for d in reports) / len(reports)
        mean_dict[label] = dictionary

    return mean_dict
def print_report(avg_reports):
    from prettytable import PrettyTable
    x = PrettyTable()

    x.field_names = ["Algorithm", "Precision", "Recall", "F1"]

    for label in avg_reports.keys():
        if label in 'accuracy':
            x.add_row(['---','---','---','---'])
            continue
        x.add_row([label, 
                   avg_reports[label]['precision'], 
                   avg_reports[label]['recall'], 
                   avg_reports[label]['f1-score']])


    print(x)
    print('Accuracy: ', avg_reports['accuracy'])
def base_level_eval(users, predictions):
    """Uses the predctions to return the average of the ndcg impact at base level.
    Args:
        users: list of users ids
        predictions:predictions for users. PREDS HAVE TO be the same index ahas the users list
    Returns:
        average of ndcg
    """
    print('starting base_level_eval')
    results_algo = pd.read_csv(os.path.join(DATA_DIR, 'results_metadataset.csv'))
    base_impact = []
    base_impact_zeroes_most = []
    base_impact_zeroes_best = []
    count_users = 0
    count_users_less= 0
    count_users_same= 0
    for user_uid, pred in zip(users, predictions):


        val = results_algo.loc[ results_algo['original_id'] == user_uid, pred ]
        if pred == 'zeroes':
            val_zeroes = results_algo.loc[ results_algo['original_id'] == user_uid, 'als_ndcg']
            best = results_algo.loc[ results_algo['original_id'] == user_uid]

            base_impact.append(val.values[0])
            base_impact_zeroes_most.append(val_zeroes.values[0])
            base_impact_zeroes_best.append(best.drop('original_id', 1).max(axis=1).values[0])
        else:
            base_impact.append(val.values[0])
            base_impact_zeroes_most.append(val.values[0])
            base_impact_zeroes_best.append(val.values[0])
    

        if len(val.values) > 1:
            raise Exception("More than one case")
        
        als_value = results_algo.loc[ results_algo['original_id'] == user_uid, 'als_ndcg' ]
        if val.values[0] > als_value.values[0]:
            if count_users == 0:
                print(val.values[0], als_value.values[0])
            count_users+=1
        elif val.values[0] == als_value.values[0]:
            if count_users_same < 5:
                print('same',val.values[0], als_value.values[0])
            count_users_same+=1
        else:
            if count_users_less == 0:
                print(val.values[0], als_value.values[0])
            count_users_less+=1
            
            
    print("%users that predicted class ndcg is better than ALS ", count_users / len(users) )
    print("%users that predicted class ndcg is same than ALS", count_users_same / len(users) )
    print("%users that predicted class ndcg is worst than ALS", count_users_less / len(users) )

    return np.mean(base_impact), np.mean(base_impact_zeroes_most), np.mean(base_impact_zeroes_best)


#### Print

In [71]:
def print_report(avg_reports):
    from prettytable import PrettyTable
    x = PrettyTable()

    x.field_names = ["Algorithm", "Precision", "Recall", "F1"]

    for label in avg_reports.keys():
        if label in 'accuracy':
            x.add_row(['---','---','---','---'])
            continue
        x.add_row([label, 
                   avg_reports[label]['precision'], 
                   avg_reports[label]['recall'], 
                   avg_reports[label]['f1-score']])


    print(x)
    print('Accuracy: ', avg_reports['accuracy'])

In [72]:
def report_average(reports):
    mean_dict = dict()
    for label in reports[0].keys():
        dictionary = dict()

        if label in 'accuracy':
            mean_dict[label] = sum(d[label] for d in reports) / len(reports)
            continue

        for key in reports[0][label].keys():
            dictionary[key] = sum(d[label][key] for d in reports) / len(reports)
        mean_dict[label] = dictionary

    return mean_dict

### Grid Search

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [10 ,75, 150],
    'max_features': ['auto', 'sqrt', 'log2'],
    'n_estimators': [100, 250, 1000],
}

In [ ]:
rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 10)
grid_search.fit(inputs, target) 

In [ ]:
grid_search.best_params_